### Import Library Python

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

### Konfigurasi Model AI

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

my_api = os.getenv("NAGA_AI_KEY")
client = OpenAI(
    api_key= my_api,
    base_url='https://api.naga.ac/v1'
    )

def generate(prompt, model="gpt-3.5-turbo", temperature=0, max_tokens=1000):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return response.choices[0].message.content

### Connect ke Database SQLite

In [3]:
conn = sqlite3.connect('amazon_sales.db')

In [4]:
df = pd.read_excel("data_source/Amazon Sales.xlsx")

In [5]:
df.to_sql('sales', conn, if_exists='replace', index=False)

1465

In [6]:
query = """
SELECT *
FROM sales
LIMIT 5
"""

pd.read_sql_query (query, conn)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,qty,rating,rating_count,order_date,arrival_date,about_product,user_id,user_name,review_id,review_title,review_content
0,B0912WJ87V,Reffair AX30 [MAX] Portable Air Purifier for C...,Car & Motorbike,2339.0,4000.0,0.42,6,3.8,1118,2003-02-24 00:00:00,2003-02-26 00:00:00,[NEW LAUNCH - INNOVATIVE DESIGN] EFFICIENT PUR...,"AG6W5HESRSDLBX3NCYOOUGFOWERA,AED4U5CF37HVSYPMD...","Arivazhagan,Ajit,swanand pandit,Venkatesh S.,S...","R3TOOFPX256D59,R1PR50BDQOEIIO,R392FI4QWXWOX5,R...","Liked it,User friendly and effective,Not as pe...","As of now its working fine, but don't expect m..."
1,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers & Accessories,399.0,1099.0,0.64,4,4.2,24269,2003-05-07 00:00:00,2003-05-17 00:00:00,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
2,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers & Accessories,199.0,349.0,0.43,6,4,43994,2003-07-01 00:00:00,2003-07-03 00:00:00,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
3,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers & Accessories,199.0,1899.0,0.90,4,3.9,7928,2003-08-25 00:00:00,2003-08-26 00:00:00,【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
4,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers & Accessories,329.0,699.0,0.53,5,4.2,94363,2003-10-10 00:00:00,2003-10-11 00:00:00,The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."


### Generate Query SQL

In [7]:
daftar_kolom = df.columns.tolist()

prompt = (
    f"saya memiliki dataset dengan kolom : {str(daftar_kolom)}"
    "Buatkan SQL query untuk menghitung total penjualan per kategori produk setiap bulan."
)
result = generate(prompt)

print(result)

Anda dapat menggunakan query SQL berikut untuk menghitung total penjualan per kategori produk setiap bulan:

```sql
SELECT category, DATE_FORMAT(order_date, '%Y-%m') AS month, SUM(qty) AS total_sales
FROM dataset_table
GROUP BY category, DATE_FORMAT(order_date, '%Y-%m')
ORDER BY category, DATE_FORMAT(order_date, '%Y-%m');
```

Pastikan untuk mengganti `dataset_table` dengan nama tabel sesuai dengan dataset Anda. Query ini akan mengelompokkan total penjualan per kategori produk setiap bulan berdasarkan tanggal pesanan.


### Analisis Query SQL yang dihasilkan oleh OpenAI

1. SELECT category, DATE_FORMAT(order_date, '%Y-%m') AS month, SUM(qty) AS total_sales: Penggunaan SELECT ini sudah benar. AI memilih kolom 'category', 'order_date', dan 'qty', dengan melakukan format ulang 'order_date' ke tahun-bulan dan menghitung jumlah 'qty' sebagai total penjualan.
2. FROM dataset_table: Ini tergantung pada nama tabel sebenarnya di dalam database.
3. GROUP BY category, DATE_FORMAT(order_date, '%Y-%m'): Penggunaan GROUP BY ini benar. AI mengelompokkan hasil berdasarkan kategori produk dan bulan order untuk menjumlahkan total penjualan.
4. ORDER BY category, DATE_FORMAT(order_date, '%Y-%m'): Penggunaan ORDER BY ini juga sudah tepat. AI mengurutkan hasil berdasarkan kategori produk dan bulan order untuk memudahkan analisis.

### Validasi Query SQL

In [9]:
query = """
SELECT category, 
    strftime('%Y-%m', order_date) AS month, 
    SUM(qty) AS total_sales
FROM 
    sales
GROUP BY 
    category, 
    strftime('%Y-%m', order_date)
ORDER BY 
    category, 
    strftime('%Y-%m', order_date);
"""
pd.read_sql_query(query, conn)

,category,month,total_sales
0,Car & Motorbike,2003-02,6
1,Computers & Accessories,2003-01,22
2,Computers & Accessories,2003-02,23
3,Computers & Accessories,2003-03,26
4,Computers & Accessories,2003-04,34
...,...,...,...
111,Office Products,2005-02,8
112,Office Products,2005-03,5
113,Office Products,2005-04,7
114,Office Products,2005-05,7


### Analisis Kesimpulan

Saya melihat bahwa kemampuan AI untuk menghasilkan query SQL merupakan contoh yang menonjol dalam bidang pengolahan bahasa alami (Natural Language Processing, NLP). Saya menganggap bahwa dengan kemajuan teknologi NLP,AI dapat memahami perintah-perintah dalam bahasa alami yang diberikan oleh pengguna dan mengonversinya menjadi query SQL yang sesuai. Hal ini memudahkan pengguna untuk mengambil data dari basis data tanpa harus memiliki pengetahuan mendalam tentang bahasa SQL.

Menurut saya, keuntungan dari penggunaan AI untuk menghasilkan query SQL adalah kemudahan penggunaan. Saya melihat bahwa pengguna tidak perlu memiliki pengetahuan teknis yang mendalam tentang SQL untuk melakukan kueri terhadap database. Mereka hanya perlu memberikan instruksi dalam bahasa alami kepada AI, yang kemudian akan menghasilkan query SQL sesuai dengan instruksi tersebut. Selain itu, saya melihat bahwa dengan bantuan AI, pengguna dapat menghasilkan query SQL dengan cepat dan efisien, menghemat waktu mereka daripada harus mempelajari sintaks SQL atau mengembangkan kueri secara manual. Hal ini menurut saya, dapat meningkatkan efisiensi dalam analisa data dari database.

Namun, saya juga melihat bahwa ada beberapa keterbatasan yang perlu diperhatikan terkait dengan penggunaan AI untuk menghasilkan query SQL. Salah satunya adalah keterbatasan pemahaman konteks. Meskipun AI dapat memahami instruksi dalam bahasa alami, saya melihat bahwa ada kemungkinan bahwa AI tidak sepenuhnya memahami konteks atau maksud yang dimaksud oleh pengguna. Hal ini dapat mengakibatkan terjadinya kesalahan atau hasil query yang tidak sesuai dengan yang diharapkan.

Saya juga melihat bahwa keterbatasan dalam memahami variasi sintaks merupakan hal yang perlu diperhatikan. Meskipun AI dapat menghasilkan query SQL yang sesuai dengan instruksi yang diberikan, saya menganggap bahwa kemampuannya untuk memahami berbagai variasi sintaks SQL mungkin terbatas. Ini, menurut saya, dapat menyebabkan AI menghasilkan query dengan sintaks yang kurang optimal atau tidak efisien.

Terakhir, menurut saya, keterbatasan pada kueri yang rumit juga perlu diperhatikan. Saya melihat bahwa AI mungkin mengalami kesulitan dalam menghasilkan query SQL untuk kueri yang kompleks atau rumit. Hal ini disebabkan oleh keterbatasan pemahaman AI terhadap logika dan hubungan antar data yang kompleks. Dengan memahami kelebihan dan keterbatasan ini, kita sebagai pengguna dapat menggunakan AI secara efektif dalam menghasilkan query SQL untuk kebutuhan pengolahan data. Namun, kita juga perlu berhati-hati dan memvalidasi hasil query yang dihasilkan untuk memastikan keakuratannya.